In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import sys
sys.path.append('C:/Users/au723200/FEPCEU')
from enhanced_experiment import EnhancedTimeSeriesExperiment
from models.arima import Arima
from models.baseline import Baseline
from models.chronos import Chronos
from models.lgbm import LGBM
from models.neuralproph import Neuralprophet

from data.data_retriever import Dataretreiver

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
data = Dataretreiver(reduce='pca')
df = data.combined

In [40]:
# Define models
models = {
    'BASELINE': Baseline(),
    #'LGBM': LGBM(),
    #'ARIMA': Arima(order=[1,1,1], seasonal_order=[1,1,1,24]),
    #'NEURALPROPHET': Neuralprophet()
}

In [41]:
# Initialize the experiment
experiment = EnhancedTimeSeriesExperiment(
    models=models,
    target_column='price',
    forecast_horizon=192, # Forecast 8 days to avoid weekly seasonality.
    n_splits=10
)


# Define custom combinations of different measurement types with different areas
custom_combinations = [
    ## For adding all wind and sun columns in PCA reduced.
    #{
    #    'wind': {22, 34, 7, 10, 16, 32, 24, 31, 1, 4},
    #},
    #{
    #    'wind': {22, 34, 7, 10, 16, 32, 24, 31, 1, 4},
    #    'sun':  {23, 8, 39, 9, 21, 31, 30, 33, 6, 20}
    #},
]

# Run the experiments with these custom combinations
experiment.run_feature_group_experiments(
    df=df,
    custom_feature_combinations=custom_combinations,
    add_all_columns=True
)

Identified area columns: {'sun': {22: 'sun_area_22', 8: 'sun_area_8', 39: 'sun_area_39', 9: 'sun_area_9', 16: 'sun_area_16', 31: 'sun_area_31', 30: 'sun_area_30', 32: 'sun_area_32', 6: 'sun_area_6', 19: 'sun_area_19'}, 'wind': {22: 'wind_area_22', 34: 'wind_area_34', 7: 'wind_area_7', 10: 'wind_area_10', 16: 'wind_area_16', 32: 'wind_area_32', 24: 'wind_area_24', 31: 'wind_area_31', 1: 'wind_area_1', 4: 'wind_area_4'}, 'temp': {22: 'temp_area_22', 2: 'temp_area_2', 26: 'temp_area_26', 40: 'temp_area_40', 15: 'temp_area_15', 28: 'temp_area_28', 24: 'temp_area_24', 37: 'temp_area_37', 5: 'temp_area_5', 36: 'temp_area_36'}}
Cross-validation splits:
Split 1: Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531)
Split 2: Train end = 2024-07-07 20:00:00 (index 4532), Test end = 2024-07-15 19:00:00 (index 4723)
Split 3: Train end = 2024-07-15 20:00:00 (index 4724), Test end = 2024-07-23 19:00:00 (index 4915)
Split 4: Train end = 2024-07-23 20:00:00 (index 4

In [42]:
# Get a table of results
results_df = experiment.summarize_feature_group_results(metric='elapsed_time')
print(results_df)

# Plot the results
experiment.plot_feature_group_results(metric='elapsed_time')

                   BASELINE
base_only  0.1310 Â± 0.0000
all_areas  0.1361 Â± 0.0000


In [43]:
# Plot predictions for the 'sun_areas_1_2_3' feature group
experiment.plot_feature_group_predictions(
    df=df,
    feature_group='custom_temp:[1]_wind:[7]_sun:[13]',
    fold_index=0  # First CV fold
)

No predictions available for model BASELINE in feature group custom_temp:[1]_wind:[7]_sun:[13], fold 0
